# Model Selection
Mistral AI provides 5 API endpoints featuring 5 leading language models.<br>
<img src="models.png" width="250" height="150">
<img src="graph.png" width="400" height="150"><br>
Looking at the model performance such as the Multitask Language Understanding (MMLU) task. Mistral-large outperforms the rest of the models in every benchmark including reasoning, multilingual tasks, mathematics and coding. However performance might not be the only consideration here.
For our applications, we might also want to consider pricing. Mistral offers competitive pricing on their models. And it’s worth considering the performance pricing trade-offs.<br>
![](comp.png)
**1. Mistral-small →** simple tasks that one can do in bulk (classification, customer support text generation).<br>
**2. Mistral-medium →** intermediate tasks that require moderate reasoning (data extraction, summarizing a document, writing emails, writing a job description, writing product descriptions).<br>
**3. Mistral-large →** complex tasks that require large reasoning capabilities or are highly specialized (synthetic text generation, code generation, RAG, Agents).<br>

In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

## Mistral Small
First we will use Mistral-small for simpler tasks like classification.

In [3]:
prompt = """
Classify the following email to determine if it is spam or not.
Only respond with the exact text "Spam" or "Not Spam". 

# Email:
🎉 Urgent! You've Won a $1,000,000 Cash Prize! 
💰 To claim your prize, please click on the link below: 
https://bit.ly/claim-your-prize
"""

Let's try to classify an email as spam or not spam.

In [4]:
mistral(prompt, model="mistral-small-latest")

'Spam'

All the models are good at such tasks but using mistral-small for such tasks is more effective and fast.

## Mistral Medium
Now we will use Mistral-medium to perform some intermediate tasks like language transformation.

In [5]:
prompt = """
Compose a welcome email for new customers who have just made 
their first purchase with your product. 
Start by expressing your gratitude for their business, 
and then convey your excitement for having them as a customer. 
Include relevant details about their recent order. 
Sign the email with "The Fun Shop Team".

Order details:
- Customer name: Anna
- Product: hat 
- Estimate date of delivery: Feb. 25, 2024
- Return policy: 30 days
"""

Here we asking the model to compose emails for new customers who have just made their first purchase with our product. Make sure we have mentioned the order details in the prompt.

In [6]:
response_medium = mistral(prompt, model="mistral-medium-latest")

In [7]:
print(response_medium)

Subject: Welcome to The Fun Shop, Anna! Thank you for your first purchase.

Dear Anna,

We are thrilled to have you as a new customer at The Fun Shop! We want to express our sincere gratitude for your recent purchase of our stylish hat. Your support means the world to us.

We are excited for you to enjoy your new hat, and we hope it brings a touch of fun and joy to your wardrobe. Your order is currently being processed, and we anticipate that it will be delivered to you by February 25, 2024. Please keep an eye out for the tracking information, which will be sent to you via email once your order has been shipped.

If for any reason you are not completely satisfied with your purchase, please know that we offer a 30-day return policy. We want to make sure that you are completely happy with your new hat, and if it doesn't meet your expectations, we are here to help.

Thank you once again for choosing The Fun Shop. If you have any questions or concerns, please don't hesitate to reach out to

## Mistral Large
Mistral-large is great for complex tasks that require advanced reasoning capabilities or that are highly specialized.

In [8]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

In this example let's ask the model to calculate the difference in payment dates between the two customers whose payment amount are close to each other in a given dataset. First let's try using mistral-small on this task.

In [9]:
response_small = mistral(prompt, model="mistral-small-latest")

In [10]:
print(response_small)

To solve this problem, first, we need to find the two customers whose payment amounts are closest to each other. From the given dataset, we can see that the payment amounts are: 125.5, 89.99, 120.0, 54.3, and 210.2.

The two closest amounts are 120.0 and 125.5. Now, we need to find the customers and their corresponding payment dates.

Customer C001 made a payment of 125.5 on 2021-10-05.
Customer C003 made a payment of 120.0 on 2021-10-07.

So, the difference in payment dates between these two customers is 2 days (2021-10-07 minus 2021-10-05).


Here we can see that the mistral-small gives incorrect answer. But since the model results are probabilistic if we actually run it multiple times it might sometimes give the correct result.
Now let's try to perform the same task on mistral-large.

In [11]:
response_large = mistral(prompt, model="mistral-large-latest")

In [12]:
print(response_large)

To solve this problem without writing code, we first need to identify the two customers whose payment amounts are closest to each other.

The payment amounts are as follows:
1. C001: 125.5, 210.2
2. C002: 89.99, 54.3
3. C003: 120.0

The closest payments are 125.5 (C001) and 120.0 (C003), with a difference of 5.5.

Next, we need to calculate the difference in payment dates between these two customers.

The payment dates are as follows:
1. C001: 2021-10-05, 2021-10-08
2. C002: 2021-10-06, 2021-10-05
3. C003: 2021-10-07

The relevant payment dates are 2021-10-05 (C001) and 2021-10-07 (C003). The difference in payment dates is 2 days.


As we can see mistral-large splits the question into multiple steps and is able to give us the right answer.

## Numerical
Let's try another example in which we will ask the models to calculate expenses on each categories like resturants, groceries, stuffed animals and props.

In [13]:
transactions = """
McDonald's: 8.40
Safeway: 10.30
Carrefour: 15.00
Toys R Us: 20.50
Panda Express: 10.20
Beanie Baby Outlet: 25.60
World Food Wraps: 22.70
Stuffed Animals Shop: 45.10
Sanrio Store: 85.70
"""

prompt = f"""
Given the purchase details, how much did I spend on each category:
1) restaurants
2) groceries
3) stuffed animals and props
{transactions}
"""

In [14]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

To categorize your purchases, we can make the following assumptions:

1) Restaurants: McDonald's and Panda Express
2) Groceries: Safeway and Carrefour
3) Stuffed animals and props: Toys R Us, Beanie Baby Outlet, Stuffed Animals Shop, and Sanrio Store

Now, let's calculate the total spent in each category:

1) Restaurants:
   McDonald's: $8.40
   Panda Express: $10.20
   Total: $18.60

2) Groceries:
   Safeway: $10.30
   Carrefour: $15.00
   Total: $25.30

3) Stuffed animals and props:
   Toys R Us: $20.50
   Beanie Baby Outlet: $25.60
   Stuffed Animals Shop: $45.10
   Sanrio Store: $85.70
   Total: $176.90

So, you spent $18.60 on restaurants, $25.30 on groceries, and $176.90 on stuffed animals and props.


Running mistral-small on this we can observe some mistakes in the output. Let's try running mistral-large.

In [15]:
response_large = mistral(prompt, model="mistral-large-latest")
print(response_large)

Based on the purchase details you provided, here's the breakdown of your spending in each category:

1) Restaurants:
   - McDonald's: $8.40
   - Panda Express: $10.20
   - World Food Wraps: $22.70
   Total spending on restaurants: $41.30

2) Groceries:
   - Safeway: $10.30
   - Carrefour: $15.00
   Total spending on groceries: $25.30

3) Stuffed animals and props:
   - Toys R Us: $20.50
   - Beanie Baby Outlet: $25.60
   - Stuffed Animals Shop: $45.10
   - Sanrio Store: $85.70
   Total spending on stuffed animals and props: $176.90


Mistral-large gives correct answers for each categories.

## Writing and checking code

In [16]:
user_message = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

Your code should pass these tests:

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
"""

In [17]:
print(mistral(user_message, model="mistral-large-latest"))

Sure, I can help you with that. Here's a Python function that should do the trick:

```python
def twoSum(nums, target):
    if len(nums) < 2:
        return "Not enough numbers in the array"

    num_dict = {}
    for i, num in enumerate(nums):
        if target - num in num_dict:
            return [num_dict[target - num], i]
        else:
            num_dict[num] = i

    return "No two numbers in the array add up to the target"
```

This function works by iterating through the array and keeping track of the numbers and their indices in a dictionary. For each number, it checks if the difference between the target and the current number is in the dictionary. If it is, it means that we have found two numbers that add up to the target, and we return their indices. If we go through the entire array and don't find any such pair, we return a message indicating that no such pair exists.

Please note that the function will return a list of indices that add up to the target. The order of the

## Natively Fluent in English, French, Spanish, German, and Italian
We can use Mistral models for more than translating from one language to another. If we are a native Spanish speaker, for instance, we can communicate with Mistral models in Spanish for any of our tasks.

In [18]:
user_message = """
Lequel est le plus lourd une livre de fer ou un kilogramme de plume
"""

In [19]:
print(mistral(user_message, model="mistral-large-latest"))

Une livre de fer et un kilogramme de plumes ont des poids différents.

Une livre est une unité de mesure utilisée principalement aux États-Unis et au Royaume-Uni, et elle est égale à environ 0,453592 kilogrammes. Donc, une livre de fer est plus légère qu'un kilogramme de plumes.

Cependant, il est important de noter que la question peut sembler trompeuse, car on pourrait penser que la réponse évidente est que le fer est plus lourd que les plumes. C'est vrai, mais seulement si l'on compare des quantités égales de chaque matériau. Dans ce cas, on compare une livre de fer à un kilogramme de plumes, ce qui n'est pas une comparaison juste.


## List of Mistral models that we can call:

We can also call the two open source mistral models via API calls.
Here is the list of models that you can try:
```
open-mistral-7b
open-mixtral-8x7b
open-mixtral-8x22b
mistral-small-latest
mistral-medium-latest
mistral-large-latest
```

For example:
```Python
mistral(prompt, model="open-mixtral-8x22b")
```